In [2]:
from datasets import load_dataset,load_from_disk,Dataset
import torch
from bert_with_adversary import BertWithAdversary
from transformers import BertForMaskedLM, BertConfig, BertTokenizer,DataCollatorForLanguageModeling,BertModel
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import re

f:\anaconda\install\envs\fairness-gym\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = BertTokenizer.from_pretrained('D:/wfy/code/model/bert_model')
model = BertModel.from_pretrained('D:/wfy/code/model/bert_model')

In [ ]:
train_dataset = load_dataset("D:/wfy/datasets/Bios", split='train')
test_dataset = load_dataset("D:/wfy/datasets/Bios", split='test')
dev_dataset = load_dataset("D:/wfy/datasets/Bios", split='dev')

In [ ]:
test = train_dataset.select(range(1))

In [ ]:
# 定义提取第一句话的函数
def extract_first_sentence(example):
    text = example['hard_text']  # 假设文本字段是 'hard_text'
    first_sentence = re.match(r'(.*?[.!?])(\s|$)', text)
    return {'sentence': first_sentence.group(1) if first_sentence else text}

# 函数：使用 tokenizer 编码文本
def tokenize_and_encode(examples):
    return tokenizer(examples['sentence'], truncation=True, padding="max_length", max_length=128)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
)

In [ ]:
test = test.map(extract_first_sentence)

In [ ]:
test = test.map(tokenize_and_encode)

In [ ]:
train_dataset = train_dataset.map(extract_first_sentence)
train_dataset = train_dataset.map(tokenize_and_encode)
test_dataset = test_dataset.map(extract_first_sentence)
test_dataset = test_dataset.map(tokenize_and_encode)

In [ ]:
train_dataset = train_dataset.remove_columns(['hard_text', 'sentence'])
test_dataset = test_dataset.remove_columns(['hard_text', 'sentence'])

In [ ]:
train_dataset.save_to_disk('D:/wfy/datasets/Bios_sentence/train_dataset')
test_dataset.save_to_disk('D:/wfy/datasets/Bios_sentence/test_dataset')

In [ ]:
a  = load_from_disk('D:/wfy/datasets/Bios_sentence/train_dataset')

In [ ]:
a = a.select(range(2))

In [ ]:
data_collator([a[0]])

In [ ]:
train_dataset = train_dataset.map(extract_first_sentence)
test_dataset = test_dataset.map(extract_first_sentence)

In [ ]:
train_dataset = train_dataset.map(tokenize_and_encode,batched=True)
test_dataset = test_dataset.map(tokenize_and_encode,batched=True)

In [ ]:
a = load_from_disk('D:/wfy/datasets/Bios_first_sentence/train_dataset')



In [ ]:
a

In [ ]:
df = pd.read_csv('D:/wfy/datasets/Bios_first_sentence/train_dataset.csv')
df.head()

In [ ]:
# 数据 collator，用于动态地创建 MLM 训练样本
train_dataset = Dataset.from_pandas(df[['input_ids','attention_mask','profession','gender']])
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer)
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=1,               # 设置批量大小
    shuffle=True,                # 在每个 epoch 混洗数据
    collate_fn=data_collator     # 使用 data collator 将数据批处理并应用 MLM 掩码
)

In [ ]:
first_batch = next(iter(train_dataloader))

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
)

In [ ]:
train_dataset = TensorDataset(
    torch.tensor(train_dataset['first_input_ids']),
    torch.tensor(train_dataset['first_attention_mask']),
    torch.tensor(train_dataset['gender']),
    torch.tensor(train_dataset['profession']),
    
)

In [ ]:
train_dataset[0][0]

In [ ]:
mlm = data_collator([train_dataset[3][0]])
mlm

In [62]:
a  = load_from_disk('D:/wfy/datasets/Bios_sentence/train_dataset')

In [64]:
a[0]['input_ids']

[101,
 2002,
 2003,
 2036,
 1996,
 2622,
 2599,
 1997,
 1998,
 2350,
 12130,
 2000,
 1996,
 2330,
 3120,
 21365,
 2099,
 1013,
 25837,
 1000,
 3733,
 2575,
 2620,
 2243,
 1012,
 1000,
 2002,
 3687,
 1037,
 3040,
 1521,
 1055,
 3014,
 1999,
 3274,
 2671,
 2013,
 1996,
 2118,
 1997,
 4174,
 1011,
 6203,
 10280,
 1010,
 2073,
 2002,
 2003,
 2036,
 2019,
 20621,
 9450,
 1012,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [4]:
# 加载 bios 数据集
dataset = load_dataset("D:/wfy/datasets/Bios")

# 查看数据集的前几条数据
print(dataset['train'])  # 输出训练集的第一个样本

Dataset({
    features: ['hard_text', 'profession', 'gender'],
    num_rows: 257478
})


In [5]:
# 函数：使用 tokenizer 编码文本
def tokenize_and_encode(examples):
    return tokenizer(examples['hard_text'], truncation=True, padding="max_length", max_length=128)

In [6]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [7]:
train_dataset=train_dataset.map(tokenize_and_encode)
test_dataset=test_dataset.map(tokenize_and_encode)

In [8]:
train_dataset.save_to_disk('D:/wfy/datasets/Bios_classification/train_dataset')
test_dataset.save_to_disk('D:/wfy/datasets/Bios_classification/test_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/257478 [00:00<?, ? examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 99069/99069 [00:00<00:00, 1345504.75 examples/s]


In [1]:
import datasets

f:\anaconda\install\envs\fairness-gym\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset_cls = datasets.load_from_disk('D:/wfy/datasets/Bios_classification/train_dataset')
train_dataset = datasets.load_from_disk('D:/wfy/datasets/Bios_sentence/train_dataset')

In [4]:
print(type(train_dataset_cls))

<class 'datasets.arrow_dataset.Dataset'>


In [13]:
print(train_dataset_cls[1]['input_ids'])
print(train_dataset[1]['input_ids'])

[101, 2016, 2003, 2583, 2000, 14358, 1010, 22939, 26745, 3366, 1998, 7438, 3576, 7355, 3785, 1998, 4654, 10732, 28483, 9285, 1997, 2070, 2146, 2744, 3785, 1012, 2014, 15644, 2421, 5068, 2236, 6821, 1010, 5065, 1997, 8329, 1010, 9827, 1999, 2740, 2671, 1010, 5057, 2729, 18742, 1998, 2981, 6821, 3653, 11020, 3089, 10472, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[101, 2016, 2003, 2583, 2000, 14358, 1010, 22939, 26745, 3366, 1998, 7438, 3576, 7355, 3785, 1998, 4654, 10732, 28483, 9285, 1997, 2070, 2146, 2744, 3785, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [51]:
train_dataset[0]

{'profession': 21,
 'gender': 0,
 'input_ids': [101,
  2002,
  2003,
  2036,
  1996,
  2622,
  2599,
  1997,
  1998,
  2350,
  12130,
  2000,
  1996,
  2330,
  3120,
  21365,
  2099,
  1013,
  25837,
  1000,
  3733,
  2575,
  2620,
  2243,
  1012,
  1000,
  2002,
  3687,
  1037,
  3040,
  1521,
  1055,
  3014,
  1999,
  3274,
  2671,
  2013,
  1996,
  2118,
  1997,
  4174,
  1011,
  6203,
  10280,
  1010,
  2073,
  2002,
  2003,
  2036,
  2019,
  20621,
  9450,
  1012,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [ ]:
from collections import defaultdict
import numpy as np

In [ ]:
def calculate_metrics_by_group(sensitive_attributes, predictions, true_labels):
    # 初始化存储每个群体的统计信息
    groups = defaultdict(lambda: {"TP": 0, "FN": 0, "FP": 0, "TN": 0, "total": 0, "correct": 0})
    
    # 遍历每个样本，根据敏感属性分组计算统计值
    for s, pred, true in zip(sensitive_attributes, predictions, true_labels):
        groups[s]["total"] += 1  # 增加样本总数
        if pred == true:
            groups[s]["correct"] += 1  # 预测正确
            if true == 1:
                groups[s]["TP"] += 1  # 真正例
            else:   
                groups[s]["TN"] += 1  # 真负例
        else:
            if true == 1:
                groups[s]["FN"] += 1  # 假负例
            else:
                groups[s]["FP"] += 1  # 假正例
    
    # 计算每个群体的公平性指标
    metrics_by_group = {}
    for group, stats in groups.items():
        tp = stats["TP"]
        fn = stats["FN"]
        fp = stats["FP"]
        tn = stats["TN"]
        total = stats["total"]
        correct = stats["correct"]
        
        # 计算 TPR、Accuracy 等指标
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
        accuracy = correct / total if total > 0 else 0
        metrics_by_group[group] = {
            "TPR": tpr,
            "Accuracy": accuracy,
            "Samples": total
        }
    
    return metrics_by_group

In [ ]:
sensitive_attributes = [0, 0, 1, 1, 0, 2]
predictions = [0, 1, 1, 0, 1, 1]
true_labels = [0, 1, 1, 0, 0, 1]

In [ ]:
metrics = calculate_metrics_by_group(sensitive_attributes, predictions, true_labels)
# for group, metric in metrics.items():
#     print(f"Group: {group}")
#     print(f"  TPR: {metric['TPR']:.4f}")
#     print(f"  Accuracy: {metric['Accuracy']:.4f}")
#     print(f"  Samples: {metric['Samples']}")

In [ ]:
metrics